In [ ]:
import dynet as dy
import numpy as np
from collections import defaultdict
from itertools import count
import sys
import random

LAYERS = 1
INPUT_DIM = 30
HIDDEN_DIM = 50

builder_type = dy.SimpleRNNBuilder
#builder = dy.LSTMBuilder
trainer_type = dy.SimpleSGDTrainer

characters = list("abcdefghijklmnopqrstuvwxyz,. ")
characters.append("<EOS>")

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)